In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Audio
import cv2
from scipy.io.wavfile import write
from datetime import datetime
import subprocess

# Simulation

In [ ]:
def get_interpolated(array, index):
    if not hasattr(index, "__len__") or len(index) < 1: return array # if scalar
    return (1 - (index[0] % 1)) * get_interpolated(array[int(np.floor(index[0]))], index[1:]) + (index[0] % 1) * get_interpolated(array[int(np.ceil(index[0]))], index[1:])

In [ ]:
get_interpolated(np.array([[1, 2],[3, 4]]), [0.5])

In [ ]:
def gaussian(x, y, n, offset, width):
    x = (x - n/2.0) / (n/2.0) - offset[0]
    y = (y - n/2.0) / (n/2.0) - offset[1]
    return np.exp(-(x*x + y*y) / (width*width)) + 0j

In [ ]:
def parabolar(x, y, n, offset, factor):
    x = (x - n/2.0) / (n/2.0) - offset[0]
    y = (y - n/2.0) / (n/2.0) - offset[1]
    return factor * (x*x + y*y)


In [ ]:
def calculate_next_psi(psi, dt, potential):
    n = psi.shape[0]

    # potential-part
    next_psi = psi * np.exp(1j * dt * potential)

    next_psi = np.fft.fft2(next_psi)

    indices = 2 * np.pi * np.min([np.arange(n), n-np.arange(n)], axis=0)
    k = indices.reshape(-1, 1)
    l = indices.reshape(1, -1)
    theta = (k*k + l*l) * dt
    next_psi *= np.exp(1j * theta)

    next_psi = np.fft.ifft2(next_psi)
    return next_psi

In [ ]:
n = 128
fps = 400
skip_frames = 20
duration = 5
simulation_speed = 0.005

slits_distance = 4
slits_width = 2
wall_height = 1e60

frames = np.empty((duration*fps, n, n), dtype=complex) # for storing the generated images

psi = np.array([[gaussian(x, y, n, offset=[-0.6, 0.0], width=0.15) for x in range(n)] for y in range(n)])
potential = np.array([[parabolar(x, y, n, offset=[0,0], factor=10000) for x in range(n)] for y in range(n)])
potential[:64-slits_distance//2-slits_width, 64] = wall_height
potential[64+slits_distance//2+slits_width:, 64] = wall_height
potential[64-slits_distance//2:64+slits_distance//2, 64] = wall_height
frames[0] = psi

#plt.pcolormesh(pow(np.abs(frames[0]), 2.0/3.0), cmap='inferno', vmin=0, vmax=1)
plt.pcolormesh(potential, vmin=0, vmax=20000)
plt.colorbar()
plt.show()


for i in range(1, fps*duration):
    psi = calculate_next_psi(psi, simulation_speed / fps, potential)
    frames[i] = psi

print("Finished simulation")

# Video

In [ ]:
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_filename = f'output/simulation_{datetime.now().strftime("%Y_%m_%d-%H_%M_%S")}.mp4'
video = cv2.VideoWriter(video_filename, fourcc, fps/skip_frames, (800, 600))
for i, frame in enumerate(frames[::skip_frames]):
    plt.figure(figsize=(8, 6))
    plot = plt.pcolormesh(pow(np.abs(frame), 2.0/3.0), cmap='inferno', vmin=0, vmax=1, animated=True)
    plt.colorbar()
    plt.title('psi')

    # Save to temporary file
    plt.savefig('temp_frame.png')
    plt.close()

    # Read the image and add to the video
    img = cv2.imread('temp_frame.png')
    video.write(img)

print(f"Video saved as {video_filename}")
video.release()

# Sonification

In [ ]:
average_listening = lambda array: np.average(array)
point_listening = lambda array: array[20, 60]

In [ ]:
listening = average_listening

dampening_per_second = 1 - 1e-12
spring_amount = 480000
min_mass = 0.00125
max_mass = 3.2

sonification_duration = 5
sonification_duration = np.min([duration, sonification_duration])

sample_rate = 44100
dampening_per_sample = 1 - pow(1 - dampening_per_second, 1 / sample_rate)

spatial_step = 4
m = n // spatial_step

In [ ]:
num_simulated_steps = sonification_duration * sample_rate

deflections = np.zeros((m, m))
speeds = np.zeros((m, m), dtype=np.float64)
forces = np.zeros((m, m), dtype=np.float64)
#speeds[excitement_point] = 1

masses = np.empty((m, m))
data = frames[0, ::spatial_step, ::spatial_step]
last_data = frames[0, ::spatial_step, ::spatial_step]


audio = np.empty(num_simulated_steps)

for sample in range(num_simulated_steps):
    
    simulation_index = int(sample / sample_rate * fps)
    last_data = data
    #data = get_interpolated(frames, [sample / sample_rate * fps])
    data = frames[simulation_index, ::spatial_step, ::spatial_step]
    
    masses = min_mass + (max_mass - min_mass) * np.abs(data)
    
    # Internal cells interaction
    #'''
    forces = np.zeros((m, m), dtype=np.float64)
    # Springs 
    forces[1:  ,  :  ] += spring_amount * (deflections[ :-1,  :  ] - deflections[1:  ,  :  ]) # Bottom
    forces[ :-1,  :  ] += spring_amount * (deflections[1:  ,  :  ] - deflections[ :-1,  :  ]) # Top
    forces[ :  , 1:  ] += spring_amount * (deflections[ :  ,  :-1] - deflections[ :  , 1:  ]) # Left
    forces[ :  ,  :-1] += spring_amount * (deflections[ :  , 1:  ] - deflections[ :  ,  :-1]) # Right
    
    # Fixed points
    #forces[::n-1, ::n-1] = 0 # fixed corners
    forces[::m-1, :] = 0 # fixed top & bottom
    forces[:, ::m-1] = 0 # fixed sides
    
    '''
    # Springs compact
    forces[1:-1, 1:-1] = spring_amount * (deflections[:-2, 1:-1] + deflections[2:, 1:-1] +
                          deflections[1:-1, :-2] + deflections[1:-1, 2:] -
                          4 * deflections[1:-1, 1:-1])
    '''

    # Add noise
    # TODO: As force or directly at speeds?
    # Forces seems to be better
    forces[1:-1, 1:-1] += np.abs(np.abs(last_data) - np.abs(data))[1:-1, 1:-1] * (2 * np.random.random((m-2, m-2)) - 1)

    # Update speeds with forces, apply dampening
    speeds += forces / masses / sample_rate
    speeds *= 1 - dampening_per_sample #/ masses
    
    deflections += speeds / sample_rate

    audio[sample] = listening(deflections)

    #'''
    if sample % (sample_rate * sonification_duration // 5) == 0:
        scale = np.max(np.abs(deflections))
        plt.pcolormesh(deflections, vmin=-scale, vmax=scale, cmap='Spectral')
        plt.colorbar()
        plt.title(f"{round(sample/sample_rate, 2) }s / {sample} samples")
        plt.show()
    #'''

In [ ]:
audio[:1000]  *= np.square(np.linspace(start=0, stop=1, num=1000, endpoint=False))
audio[-1000:] *= np.square(np.linspace(start=1, stop=0, num=1000, endpoint=False))
audio_filename = f'output/sonification_{datetime.now().strftime("%Y_%m_%d-%H_%M_%S")}.wav'
write(audio_filename, sample_rate, np.round((audio - np.average(audio)) / np.max(np.abs(audio - np.average(audio))) * 32767).astype(np.int16))
print(f"Sonification saved as {audio_filename}")
Audio(audio, rate=sample_rate)

In [ ]:
plt.plot(audio)

# Combine Audio & Video

In [ ]:
combined_filename = f'output/combination_{datetime.now().strftime("%Y_%m_%d-%H_%M_%S")}.mp4'

# Construct the ffmpeg command to combine video and audio
ffmpeg_command = [
    './ffmpeg',
    '-i', video_filename,   # Input video file
    '-i', audio_filename,   # Input audio file
    '-c:v', 'copy',         # Copy the video stream
    '-c:a', 'aac',          # Encode the audio to AAC (necessary for some formats)
    '-shortest',            # Finish encoding when the shortest input stream ends
    combined_filename         # Output file
]

# Execute the command
subprocess.run(ffmpeg_command)